In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from signals import *
from frequencyestimator import *
from scipy.optimize import basinhopping
from tqdm.auto import tqdm

sns.set_style("whitegrid")
sns.despine(left=True, bottom=True)
sns.set_context("poster", font_scale = .45, rc={"grid.linewidth": 0.8})

/Users/farrokhlabib/Documents/github/csAE/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<Figure size 640x480 with 0 Axes>

In [2]:
P0 = lambda n, theta: np.cos((2*n+1)*theta)**2
P1 = lambda n, theta: np.sin((2*n+1)*theta)**2

def estimate_signal(depths, n_samples, theta, eta=0.0):
        signals = np.zeros(len(depths), dtype = np.complex128)
        measurements = np.zeros(len(depths))
        cos_signal = np.zeros(len(depths), dtype = np.complex128)
        for i,n in enumerate(depths):
            # Get the exact measuremnt probabilities
            p0 = P0(n, theta)
            p1 = P1(n, theta)

            p0x = P0x(n,theta)
            p1x = P1x(n,theta)

            # Get the "noisy" probabilities by sampling and adding a bias term that pushes towards 50/50 mixture
            eta_n = (1.0-eta)**(n+1) # The error at depth n increases as more queries are implemented
            p0_estimate = np.random.binomial(n_samples[i], eta_n*p0 + (1.0-eta_n)*0.5)/n_samples[i]
            p1_estimate = 1 - p0_estimate

            p0x_estimate = np.random.binomial(n_samples[i], eta_n*p0x + (1.0-eta_n)*0.5)/n_samples[i]
            p1x_estimate = 1.0 - p0x_estimate
            measurements[i] = p0_estimate
            
            # Estimate theta
            theta_estimated = np.arctan2(p0x_estimate - p1x_estimate, p0_estimate - p1_estimate)
            
            # Store this to determine angle at theta = 0 or pi/2
            if i==0:
                p0mp1 = p0_estimate - p1_estimate

            # Compute f(n) - Eq. 3
            fi_estimate = np.exp(1.0j*theta_estimated)
            signals[i] = fi_estimate
         
        return signals, measurements

from scipy.stats import binom

def apply_correction(ula_signal, measurements, theta_est, theta):
    p_exact = np.cos((2*ula_signal.depths+1)*(theta))**2
    p_o2 = np.cos((2 * ula_signal.depths + 1) * (theta_est/2.0)) ** 2
    p_o4 = np.cos((2 * ula_signal.depths + 1) * (theta_est/4.0)) ** 2
    p_same = np.cos((2*ula_signal.depths+1)*(theta_est))**2
    p_s2 = np.cos((2 * ula_signal.depths + 1) * (np.pi/2-theta_est)) ** 2
    p_s4 = np.cos((2 * ula_signal.depths + 1) * (np.pi / 4 - theta_est)) ** 2
    p_s2_o2 = np.cos((2 * ula_signal.depths + 1) * (np.pi / 2 - theta_est/2)) ** 2
    p_s4_o2 = np.cos((2 * ula_signal.depths + 1) * (np.pi / 4 - theta_est/2)) ** 2

    l_exact = np.sum(
        np.log([1e-75 + binom.pmf(ula_signal.n_samples[kk] * measurements[kk], ula_signal.n_samples[kk],
                                  p_exact[kk]) for kk in
                range(len(ula_signal.n_samples))]))

    l_o2 = np.sum(
        np.log([1e-75+binom.pmf(ula_signal.n_samples[kk]*measurements[kk], ula_signal.n_samples[kk], p_o2[kk]) for kk in
         range(len(ula_signal.n_samples))]))
    l_o4 = np.sum(
        np.log([1e-75+binom.pmf(ula_signal.n_samples[kk] * measurements[kk], ula_signal.n_samples[kk], p_o4[kk]) for kk in
         range(len(ula_signal.n_samples))]))
    l_same = np.sum(
        np.log([1e-75+binom.pmf(ula_signal.n_samples[kk] * measurements[kk], ula_signal.n_samples[kk], p_same[kk]) for kk in
         range(len(ula_signal.n_samples))]))
    l_s2 = np.sum(
        np.log([1e-75+binom.pmf(ula_signal.n_samples[kk] * measurements[kk], ula_signal.n_samples[kk], p_s2[kk]) for kk in
         range(len(ula_signal.n_samples))]))
    l_s4 = np.sum(
        np.log([1e-75+binom.pmf(ula_signal.n_samples[kk] * measurements[kk], ula_signal.n_samples[kk], p_s4[kk]) for kk in
         range(len(ula_signal.n_samples))]))
    l_s2_o2 = np.sum(
        np.log([1e-75+binom.pmf(ula_signal.n_samples[kk] * measurements[kk], ula_signal.n_samples[kk], p_s2_o2[kk]) for kk in
         range(len(ula_signal.n_samples))]))
    l_s4_o2 = np.sum(
        np.log([1e-75+binom.pmf(ula_signal.n_samples[kk] * measurements[kk], ula_signal.n_samples[kk], p_s4_o2[kk]) for kk in
         range(len(ula_signal.n_samples))]))
    # print(f'theta_exact obj:           {l_exact}\n')

    # print(f'2*theta_found obj:         {l_same}')
    # print(f'2*theta found:             {2*theta_est / np.pi}\n')

    # print(f'pi-2*theta_found obj:      {l_s2}')
    # print(f'pi-2*theta found:          {1.0-2 * theta_est / np.pi}\n')

    # print(f'pi/2-2*theta_found obj:    {l_s4}')
    # print(f'pi/2-2*theta found:        {0.5 - 2 * theta_est / np.pi}\n')

    # print(f'theta_found obj:           {l_o2}')
    # print(f'theta found:               {theta_est / np.pi}\n')

    # print(f'theta_found/2 obj:         {l_o4}')
    # print(f'theta found:               {theta_est / 2.0 / np.pi}\n')

    # print(f'pi - theta_found obj:    {l_s2_o2}')
    # print(f'pi - theta found:        {1.0 - theta_est / np.pi}\n')

    # print(f'pi/2 - theta_found obj:    {l_s4_o2}')
    # print(f'pi/2 - theta found:        {0.5 - theta_est / np.pi}\n')

    
    which_correction = np.argmax([l_same, l_s2, l_s4, l_o2, l_o4, l_s2_o2, l_s4_o2])
    if which_correction == 1:
        theta_est = np.pi/2.0 - theta_est
    elif which_correction == 2:
        theta_est = np.pi/4.0 - theta_est
    elif which_correction == 3:
        theta_est = theta_est/2
    elif which_correction == 4:
        theta_est = theta_est/4
    elif which_correction == 5:
        theta_est = np.pi / 2.0 - 0.5 * theta_est
    elif which_correction == 6:
        theta_est = np.pi / 4.0 - 0.5 * theta_est

    # print(f'FINAL ANGLE FOUND: {theta_est, theta}')

    return theta_est

## Use Basinhopping (simulated annealing) from scipy.optimize to minimize the objective function

In [3]:
def objective_function(lp, cos_signal, abs_sin, ula_signal, esprit):   
    signal = cos_signal + 1.0j * lp * abs_sin
    R = ula_signal.get_cov_matrix_toeplitz(signal)
    _, _ = esprit.estimate_theta_toeplitz(R)
    eigs = np.abs(esprit.eigs)[:2]
    obj = eigs[1] - eigs[0]

    return obj

## Function to implement the basinhopping based csAE

In [4]:
a = 0.7
theta = np.arcsin(a)

narray = [2,2,2,2,2,2,2,3]
ula_signal = TwoqULASignal(M=narray, C=3)
esprit = ESPIRIT()
# narray = [2]*8

def csae_with_basinhopping(theta, ula_signal, esprit, niter, T=1.0,disp=False):

    depths = ula_signal.depths
    n_samples = ula_signal.n_samples
    csignal, measurements = estimate_signal(depths, n_samples, theta)

    cos_signal = np.real(csignal)

    correct_signs = np.sign(np.imag(csignal))
    abs_sin = np.abs(np.imag(csignal))

    bounds = tuple((-1.0, 1.0) for _ in range(len(correct_signs)))
    init_signs = np.array([1.0 for _ in range(len(correct_signs))])
    # init_signs = (-1)**np.random.binomial(1, 0.5, len(correct_signs))
    res = basinhopping(objective_function, init_signs, niter=niter, T=T, stepsize=2.0, minimizer_kwargs={'args': (cos_signal, abs_sin, ula_signal, esprit), 'bounds': bounds}, disp=disp)

    if disp:
        print(res)

    x = res.x
    signal = cos_signal + 1.0j * x * abs_sin
    R = ula_signal.get_cov_matrix_toeplitz(signal)
    theta_est, _ = esprit.estimate_theta_toeplitz(R)
    theta_est = apply_correction(ula_signal, measurements, theta_est, theta) #apply correction

    if disp:
        cR = ula_signal.get_cov_matrix_toeplitz(csignal)
        _, _ = esprit.estimate_theta_toeplitz(cR)
        eigs = np.abs(esprit.eigs)[:2]
        obj = eigs[1] - eigs[0]
        print(f'true obj: {obj}')

    num_queries = np.sum(np.array(ula_signal.depths)*np.array(ula_signal.n_samples)) + ula_signal.n_samples[0]
    # Compute the maximum single query
    max_single_query = np.max(ula_signal.depths)

    ret_dict = {'theta_est': theta_est, 'error': np.abs(np.sin(theta)-np.sin(theta_est)), 'queries': num_queries, 'depth': max_single_query}
    
    return ret_dict

csae_with_basinhopping(theta=theta, narray=narray, niter=20, disp=True)

basinhopping step 0: f -92.6752
basinhopping step 1: f -601.663 trial_f -601.663 accepted 1  lowest_f -601.663
found new global minimum on step 1 with function value -601.663
basinhopping step 2: f -601.663 trial_f -115.311 accepted 0  lowest_f -601.663
basinhopping step 3: f -601.663 trial_f -111.922 accepted 0  lowest_f -601.663
basinhopping step 4: f -601.663 trial_f -250.258 accepted 0  lowest_f -601.663
basinhopping step 5: f -601.663 trial_f -601.663 accepted 1  lowest_f -601.663
found new global minimum on step 5 with function value -601.663
basinhopping step 6: f -601.663 trial_f -601.663 accepted 1  lowest_f -601.663
found new global minimum on step 6 with function value -601.663
basinhopping step 7: f -601.663 trial_f -217.259 accepted 0  lowest_f -601.663
basinhopping step 8: f -601.663 trial_f -200.983 accepted 0  lowest_f -601.663
basinhopping step 9: f -601.663 trial_f -601.663 accepted 1  lowest_f -601.663
basinhopping step 10: f -601.663 trial_f -212.906 accepted 0  low

{'theta_est': -0.7742126944246284,
 'error': 1.3991533908819185,
 'queries': 3069,
 'depth': 256}

## Let's do some statistics now

In [5]:
avals = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
avals = [0.2, 0.7, 0.9]
# avals = [0.5, 0.6, 0.7, 0.8, 0.9]
narray = [2,2,2,2,2,2,2,3]

num_queries = np.zeros(len(avals), dtype=int)
max_single_query = np.zeros(len(avals), dtype=int)

num_mc=100
thetas = np.zeros((len(avals), num_mc))
errors = np.zeros((len(avals), num_mc))

ula_signal = TwoqULASignal(M=narray, C=3)
esprit = ESPIRIT()

for j,a in enumerate(avals):
    theta = np.arcsin(a)
    print(f'theta = {theta}')
    for i in tqdm(range(num_mc)):
        res = csae_with_basinhopping(theta=theta, narray=narray, niter=10, T=1.0)
        thetas[j][i] = res['theta_est']
        errors[j][i] = res['error']
    num_queries[j] = res['queries']
    max_single_query[j] = res['depth']

    print(f'constant factors query and depth: {np.percentile(errors[j], 68) * num_queries[j]}, {np.percentile(errors[j], 68) * max_single_query[j]}')




theta = 0.2013579207903308


100%|██████████| 100/100 [1:55:12<00:00, 69.12s/it] 


constant factors query and depth: 4.830492494657762, 0.4029345319753624
theta = 0.775397496610753


100%|██████████| 100/100 [1:35:26<00:00, 57.26s/it]


constant factors query and depth: 4294.8840818846775, 358.2568670454472
theta = 1.1197695149986342


100%|██████████| 100/100 [1:33:01<00:00, 55.81s/it]

constant factors query and depth: 3.2030608518404167, 0.2671826582180341


In [6]:
errors68 = np.percentile(errors, 68, axis=1)
print(f'constant factors for query complexity: {errors68*num_queries}'),
print(f'constant factors for max depth complexity: {errors68*max_single_query}')

constant factors for query complexity: [4.83049249e+00 4.29488408e+03 3.20306085e+00]
constant factors for max depth complexity: [4.02934532e-01 3.58256867e+02 2.67182658e-01]


In [7]:
thetas

array([[ 0.20130273,  0.20075566,  0.20122481,  0.20292069,  0.20157227,
         0.20249675,  0.20143038,  0.20135112,  0.18799623,  0.18739348,
         0.20304722,  0.1890216 ,  0.19052875,  0.20237372,  0.20172074,
         0.18841337,  0.20280515,  0.20128865,  0.20280586,  0.20159018,
         0.20157867,  0.18917508,  0.20144799,  0.20177314,  0.18859687,
         0.20378216,  0.18878747,  0.20236222,  0.18886828,  0.20104689,
         0.2028963 ,  0.20286671,  0.19054771,  0.20286613,  0.20202784,
         0.20164937,  0.20364175,  0.18902302,  0.20251124,  0.2033817 ,
         0.17616168,  0.18894831,  0.20131008,  0.20308522,  0.20294281,
         0.20124626,  0.20133395,  0.20194252,  0.20281025,  0.20262044,
         0.18924581,  0.20124166,  0.1878438 ,  0.1888034 ,  0.20224058,
         0.20249774,  0.20109941,  0.20334118,  0.20279785,  0.20281336,
         0.18882544,  0.20292251,  0.20267648,  0.20066968, -0.18749986,
         0.19968006,  0.20226703,  0.20072409,  0.2

In [8]:
np.arcsin(avals)

array([0.20135792, 0.7753975 , 1.11976951])

## How does the energy landscape actually look like? 